#**Image Understanding with GPT-4.1-mini**

get your free api key from here to upload image and create URL :  https://api.imgbb.com/

##**0. Install Dependencies**

In [20]:
!pip install openai gradio pillow

##**1. Import Statements**

In [41]:
from langchain_core.tools import tool
from google.colab import userdata
# Retrieve the API key from Colab's secrets
api_key = userdata.get('openai_api_key')
imgbb_api_key = userdata.get('imgbb_api_key')

In [6]:
import gradio as gr
import tempfile
import requests
from openai import OpenAI

##**2. Create OpenAI Client**

In [42]:
client = OpenAI(api_key=api_key)

##**Optional : Response Generation**

In [17]:


response = client.responses.create(
    model="gpt-4.1-mini",
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": "what's in this image?"},
            {
                "type": "input_image",
                "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
            },
        ],
    }],
)

print(response.output_text)

This image depicts a wooden boardwalk path running through a lush green meadow with tall grasses on either side. In the distance, there are trees and bushes under a partly cloudy sky with streaks of white clouds against a blue backdrop. The scene gives a sense of calm and natural beauty, likely a park or nature reserve.


##**3. Function for Response Generation**

In [43]:
def ask_image_question(image, question):
    # Save to temp file
    with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmp:
        image.save(tmp.name)
        tmp_path = tmp.name

    # Upload to ImgBB
    with open(tmp_path, "rb") as f:
        response = requests.post(
            "https://api.imgbb.com/1/upload",
            params={"key": imgbb_api_key},
            files={"image": f},
        )

    if response.status_code != 200:
        return f"ImgBB upload failed: {response.text}"

    try:
        image_url = response.json()["data"]["url"]
    except Exception as e:
        return f"Failed to parse ImgBB response: {str(e)}"

    print("Uploaded image URL:", image_url)

    # Prepare message
    user_msg = {
        "role": "user",
        "content": [
            {"type": "input_text", "text": question},
            {"type": "input_image", "image_url": image_url}
        ]
    }

    # Call Vision model
    try:
        response = client.responses.create(
            model="gpt-4.1-mini",
            input=[user_msg]
        )
        return response.output_text
    except Exception as e:
        return f"OpenAI API error: {str(e)}"

##**4. Gradio Interface**

In [44]:
# Gradio Interface
iface = gr.Interface(
    fn=ask_image_question,
    inputs=[
        gr.Image(type="pil", label="Upload an image"),
        gr.Textbox(label="Your question about the image")
    ],
    outputs=gr.Textbox(label="GPT-4 Vision answer"),
    title="🖼️ Ask GPT-4 Vision About Your Image",
    description="Upload an image and ask GPT-4 Vision a question about it.",
)

if __name__ == "__main__":
    iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ad38569ded2709804c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
